In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as datetime
from datetime import datetime as dt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

sns.set_style('whitegrid')

# Load files

In [2]:
df_stats = pd.read_csv('../../Dataset/Dataset_clean/stats_UtoD.csv')
df_value = pd.read_csv('../../Dataset/value.csv')
df_pred = pd.read_csv('../../Dataset/Dataset_clean/stats_pred_SVM.csv')
df_stats = df_stats.drop (['Unnamed: 0'], axis = 1)

# To keep only the date without the hours and minutes
df_stats['Date'] = pd.to_datetime(df_stats['Date']) 
df_stats['Date'] = df_stats['Date'].dt.date
# Convert the date to Ordinal
df_stats['Date'] = df_stats['Date'].map(dt.toordinal)

# Drop company 'VRSN' due to limited data
df_stats = df_stats[df_stats.Ticker != 'VRSN']
df_stats_original = df_stats.copy()
df_stats_original

,Date,Ticker,Price,DE_Ratio,Trailing_P/E,Price/Sales,Price/Book,Profit_Margin,Operating_Margin,Return_on_Assets,...,Diluted_EPS,Earnings_Growth,Revenue_Growth,Total_Cash,Total_Cash_Per_Share,Total_Debt,Current_Ratio,Book_Value_Per_Share,Cash_Flow,Beta
0,731556,AAPL,1.4306,0.072,112.70,1.25,1.84,1.10,0.02,1.06,...,0.185,NaN,7.1,4.570000e+09,12.450,3.040000e+08,2.498,11.520000,NaN,1.742
1,731614,AAPL,1.5056,0.070,60.16,1.23,1.92,2.03,1.63,2.08,...,0.371,6.2,8.1,4.790000e+09,12.990,3.020000e+08,2.531,11.740000,NaN,1.749
2,731735,AAPL,1.9209,2.921,63.70,1.49,2.33,2.36,2.31,2.54,...,0.454,6.2,8.1,4.590000e+09,12.090,2.921000e+00,2.921,12.042000,3.610000e+08,1.701
3,731803,AAPL,2.1320,2.917,54.40,1.51,2.38,2.76,3.00,3.09,...,0.557,6.2,8.1,4.970000e+09,12.800,2.917000e+00,2.917,12.493000,3.610000e+08,1.744
4,731866,AAPL,2.5899,2.917,69.28,1.98,3.13,2.76,3.00,3.09,...,0.557,6.2,8.1,4.970000e+09,12.800,2.917000e+00,2.917,12.493000,5.640000e+08,1.802
5,731919,AAPL,4.5852,2.632,88.41,3.08,5.03,3.33,3.94,3.86,...,0.709,6.2,8.1,5.460000e+09,13.960,2.632000e+00,2.632,12.968000,5.640000e+08,1.775
6,731972,AAPL,4.8869,2.581,57.83,2.99,5.03,5.20,6.71,6.62,...,1.244,300.0,33.4,6.450000e+09,15.940,2.581000e+00,2.581,14.312000,9.340000e+08,1.810
7,731981,AAPL,5.3326,2.581,62.55,3.30,5.56,5.20,6.71,6.62,...,1.240,300.0,33.4,6.450000e+09,15.780,2.581000e+00,2.581,14.312000,9.340000e+08,1.810
8,732006,AAPL,6.0198,2.581,72.01,1.86,3.11,5.20,6.71,6.63,...,0.620,300.0,33.4,6.450000e+09,7.890,2.581000e+00,2.581,14.312000,1.490000e+09,1.800
9,732297,AAPL,9.7413,2.956,45.64,4.37,8.07,9.58,11.84,10.52,...,1.560,305.7,56.5,8.260000e+09,9.802,2.956000e+00,2.956,8.941000,2.540000e+09,NaN


In [3]:
df_pred.set_index('Ticker', inplace = True)
df_pred

,Unnamed: 0,Trailing_P/E,Price/Sales,Profit_Margin,Return_on_Assets,Market_Cap,Enterprise_Value/Revenue,Enterprise_Value/EBITDA,Revenue,Gross_Profit,EBITDA,Net_Income_Avl_to_Common_,Earnings_Growth,Total_Cash_Per_Share,Total_Cash,Total_Debt,Cash_Flow
Ticker,,,,,,,,,,,,,,,,,
AAPL,0,-38.277014,5.159513,42.144206,37.198930,6.771921e+11,4.916039,-11.229735,4.916039e+00,5.799914e+10,-1.122973e+01,4.763085e+10,17.064434,4.750990e+01,5.011167e+10,-6.295690e+07,3.400736e+11
ABC,1,18.780156,0.131965,0.511143,5.934260,1.312057e+10,0.084402,8.261731,8.440197e-02,2.275266e+09,8.261731e+00,7.971057e+08,-30.437188,6.048588e+00,2.485977e+09,6.611857e+08,1.515453e+10
ABT,2,12.365525,0.999473,9.945320,4.834734,1.183437e+11,1.247492,4.189305,1.247492e+00,3.112310e+10,4.189305e+00,6.589385e+09,86.170980,8.753687e+00,1.366475e+10,2.777619e+10,1.036569e+11
ADBE,3,-4.086664,-2.005349,20.461250,-11.486994,1.532514e+10,-1.818009,-5.146219,-1.818009e+00,5.724049e+09,-5.146219e+00,1.585475e+09,59.451377,4.228932e+00,3.985744e+09,2.778094e+09,2.140338e+10
ADI,4,-40.053505,-4.318554,41.781172,18.314315,-9.035958e+09,-3.994976,-27.698102,-3.994976e+00,2.278030e+09,-2.769810e+01,8.739000e+08,-107.321211,5.369828e+00,6.223645e+08,1.686219e+09,5.985791e+09
ADM,5,-5.279942,0.307949,5.404753,8.024333,4.827656e+10,0.472392,6.165325,4.723923e-01,6.322732e+09,6.165325e+00,4.137422e+09,86.195459,2.562380e+00,1.648031e+09,1.809663e+10,3.816885e+10
ADSK,6,27.864685,2.550974,11.741073,3.671546,7.859146e+09,1.744847,-26.120440,1.744847e+00,3.041136e+09,-2.612044e+01,4.067405e+08,-131.213940,8.564140e+00,2.184518e+09,4.775399e+07,8.077965e+08
AEE,7,4.291860,-0.881363,3.064406,4.048003,1.481778e+10,0.470186,3.506506,4.701862e-01,4.565273e+09,3.506506e+00,6.647378e+08,31.113100,1.466819e+00,4.824189e+08,1.357210e+10,2.975854e+09
AEP,8,2.311464,1.596299,17.395478,5.287610,5.766605e+10,3.090335,7.263521,3.090335e+00,8.678432e+09,7.263521e+00,2.191049e+09,-559.241153,-5.383533e-01,-5.182685e+07,2.248494e+10,4.174453e+10


# Problem Definition

### Apply Random Forest model to predict important characteristics investors consider when investing

In [4]:
# Date Conversion
date_pred = dt.strptime('2018-06-07', '%Y-%m-%d').date()
date_pred = date_pred.toordinal()
date_pred

736852

In [5]:
columns = ['Ticker', 'Enterprise_Value', 'Forward_P/E', 'Revenue_Growth']
dfs_result = pd.DataFrame(columns = columns)


# Feature Engineering + Model Training

In [6]:
# Start the model
model = RandomForestRegressor(n_estimators=10)
for num in range (0, 3):
    if(num == 0):
        # To predict Enterprise Value
        X_columns = ['Date', 'Market_Cap', 'Revenue', 'Gross_Profit','Net_Income_Avl_to_Common_', 'Total_Cash', 'Total_Debt', 'Cash_Flow']
        y_column = ['Enterprise_Value']
        dfs_EV = pd.DataFrame(columns = ['Ticker'] + X_columns + ['MAE', 'RMSE'])
    elif(num == 1):
        # To predict Forward_P/E
        X_columns = ['Date', 'Trailing_P/E', 'Price/Sales', 'Return_on_Assets', 'Enterprise_Value/Revenue', 'Total_Cash_Per_Share']
        y_column = ['Forward_P/E']
        dfs_FPE = pd.DataFrame(columns = ['Ticker'] + X_columns + ['MAE', 'RMSE'])
    elif(num == 2):
        # To predict Revenue Growth
        X_columns = ['Date','Trailing_P/E', 'Price/Sales', 'Profit_Margin', 'Enterprise_Value/Revenue', 'Earnings_Growth']
        y_column = ['Revenue_Growth']
        dfs_RG = pd.DataFrame(columns = ['Ticker'] + X_columns + ['MAE', 'RMSE'])
    # Make copy X_columns to data_pred
    data_pred = list(X_columns)
    
    # Adjust the dataframe
    columns_kept = ['Ticker'] + X_columns + y_column
    df_stats = df_stats_original.copy()
    df_stats = df_stats[columns_kept]
    df_stats = df_stats.dropna()
    df_stats
    
    #Dataframe for MAE and RMSE
    
    index = 0 
    for stock in df_stats['Ticker'].unique():
        # Filtering the dataframe
        df_filter = df_stats[df_stats['Ticker']==stock].copy()


        # Caluclate the required value to split the dataset
        head_split = int(len(df_filter)*0.8)
        tail_split = len(df_filter) - head_split
        # Define the training-testing set
        X_train = df_filter.head(head_split)[X_columns]
        X_test = df_filter.tail(tail_split)[X_columns]
        y_train = df_filter.head(head_split)[y_column]
        y_test = df_filter.tail(tail_split)[y_column]
        
        i = 0
        for name in X_columns:
            if(name == 'Date'):
                data_pred[i] = date_pred
            else:
                data_pred[i] = df_pred.loc[stock, name]
            i+=1
        # AAppend the to-be-predicted X_columns to X_test
        X_test.loc[100] = data_pred

        if(y_train.empty): 
            continue

        # Compute the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        prediction = y_pred[-1]
        y_pred = y_pred[:-1]
        
        #Store error values in different dataframe
        if (num == 0):
            # Enter stock name
            dfs_EV.loc[index, ['Ticker']] = stock
            # Enter Important Feature
            for i in range(len(X_columns)):
                dfs_EV.loc[index, [X_columns[i]]] = model.feature_importances_[i].round(4)
            # Enter Error
            dfs_EV.loc[index, 'MAE'] = mean_absolute_error(y_test, y_pred)
            dfs_EV.loc[index, 'RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
        elif (num == 1):
            # Enter stock name
            dfs_FPE.loc[index, ['Ticker']] = stock
            # Enter Important Feature
            for i in range(len(X_columns)):
                dfs_FPE.loc[index, [X_columns[i]]] = model.feature_importances_[i].round(4)
            # Enter Error
            dfs_FPE.loc[index, 'MAE'] = mean_absolute_error(y_test, y_pred)
            dfs_FPE.loc[index, 'RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
        elif (num == 2):
            # Enter stock name
            dfs_RG.loc[index, ['Ticker']] = stock
            # Enter Important Feature
            for i in range(len(X_columns)):
                dfs_RG.loc[index, [X_columns[i]]] = model.feature_importances_[i].round(4)
            # Enter Error
            dfs_RG.loc[index, 'MAE'] = mean_absolute_error(y_test, y_pred)
            dfs_RG.loc[index, 'RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Store values in the main dataframe
        dfs_result.loc[index, ['Ticker']] = stock
        dfs_result.loc[index, y_column] = prediction

        index += 1
        

C:\Users\Jackson\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:60: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [7]:
dfs_result

,Ticker,Enterprise_Value,Forward_P/E,Revenue_Growth
0,AAPL,4.95902e+11,11.636,33.38
1,ABC,8.905e+09,14.28,8.03
2,ABT,9.044e+10,10.592,11.54
3,ADBE,1.3851e+10,10.87,19.83
4,ADI,1.0117e+10,16.176,6.88
5,ADM,3.199e+10,9.828,43.67
6,ADSK,6.05e+09,14.457,13.8
7,AEE,1.4663e+10,15.382,13.3
8,AEP,3.5702e+10,11.519,5.24
9,AFL,2.6166e+10,13.39,6.76


In [26]:
dfs_RG[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).head(3)

,Ticker,MAE,RMSE
189,LUK,561.393,680.366
55,CHK,98.8175,106.052
267,RSG,74.395,101.444


In [29]:
dfs_RG[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).tail(3)

,Ticker,MAE,RMSE
289,TEL,0.6,0.6
253,PNW,0.6,0.6
229,NWL,0.56,0.671193


In [27]:
dfs_EV[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).head(3)

,Ticker,MAE,RMSE
38,BK,6.13713e+10,6.16669e+10
10,AIG,4.3342e+10,4.95666e+10
20,AMZN,3.71294e+10,4.13448e+10


In [30]:
dfs_EV[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).tail(3)

,Ticker,MAE,RMSE
125,FLIR,1.46667e+08,1.49164e+08
156,HUM,1.06667e+08,1.37531e+08
136,GRMN,7.15e+07,9.64909e+07


In [28]:
dfs_FPE[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).head(3)

,Ticker,MAE,RMSE
218,NFLX,91.7293,91.7803
264,RRC,19.2465,19.5962
52,CELG,15.405,15.405


In [31]:
dfs_FPE[['Ticker', 'MAE', 'RMSE']].sort_values(['MAE'], ascending = False).tail(3)

,Ticker,MAE,RMSE
233,PAYX,0.108875,0.180666
134,GPC,0.049,0.049
222,NSC,0.0303333,0.0361156


In [9]:
dfs_result.to_csv('../../Dataset/Dataset_clean/stats_pred_RF.csv')
dfs_EV.to_csv('../../Dataset/Model_Error_Examintation/RF_EV_Error.csv')
dfs_FPE.to_csv('../../Dataset/Model_Error_Examintation/RF_FPE_Error.csv')
dfs_RG.to_csv('../../Dataset/Model_Error_Examintation/RF_RG_Error.csv')

# Merge the table to the "Sharpe Ratio" table

In [10]:
df_value

,Unnamed: 0,Ticker,Expected_Annual_Return,Stock_Volatility,Sharpe_Ratio,Beta,Risk_Free_Return
0,0,AAPL,12.1920,21.2467,0.5343,0.87,6.5647
1,1,ABC,7.3741,13.4118,0.4872,0.66,5.6415
2,2,ABT,1.7563,6.8907,0.1330,0.50,4.9381
3,3,ADBE,9.4193,15.2536,0.5624,1.32,8.5430
4,4,ADI,5.8566,11.8437,0.4236,1.06,7.4000
5,5,ADM,2.0280,9.0006,0.1320,0.76,6.0811
6,6,ADSK,6.9136,14.6755,0.4139,2.15,12.1919
7,7,AEE,3.5326,6.7706,0.3977,0.23,3.7511
8,8,AEP,4.0074,8.4018,0.3770,0.34,4.2347
9,9,AFL,3.1507,12.7469,0.1813,1.66,10.0377


In [11]:
dfs_result

,Ticker,Enterprise_Value,Forward_P/E,Revenue_Growth
0,AAPL,4.95902e+11,11.636,33.38
1,ABC,8.905e+09,14.28,8.03
2,ABT,9.044e+10,10.592,11.54
3,ADBE,1.3851e+10,10.87,19.83
4,ADI,1.0117e+10,16.176,6.88
5,ADM,3.199e+10,9.828,43.67
6,ADSK,6.05e+09,14.457,13.8
7,AEE,1.4663e+10,15.382,13.3
8,AEP,3.5702e+10,11.519,5.24
9,AFL,2.6166e+10,13.39,6.76


In [12]:
df_final = pd.concat([df_value, dfs_result], axis = 1)
df_final = df_final.drop(['Unnamed: 0'], axis = 1)
df_final = df_final.loc[:,~df_final.columns.duplicated()]
df_final = df_final.dropna()
df_final

,Ticker,Expected_Annual_Return,Stock_Volatility,Sharpe_Ratio,Beta,Risk_Free_Return,Enterprise_Value,Forward_P/E,Revenue_Growth
0,AAPL,12.1920,21.2467,0.5343,0.87,6.5647,4.95902e+11,11.636,33.38
1,ABC,7.3741,13.4118,0.4872,0.66,5.6415,8.905e+09,14.28,8.03
2,ABT,1.7563,6.8907,0.1330,0.50,4.9381,9.044e+10,10.592,11.54
3,ADBE,9.4193,15.2536,0.5624,1.32,8.5430,1.3851e+10,10.87,19.83
4,ADI,5.8566,11.8437,0.4236,1.06,7.4000,1.0117e+10,16.176,6.88
5,ADM,2.0280,9.0006,0.1320,0.76,6.0811,3.199e+10,9.828,43.67
6,ADSK,6.9136,14.6755,0.4139,2.15,12.1919,6.05e+09,14.457,13.8
7,AEE,3.5326,6.7706,0.3977,0.23,3.7511,1.4663e+10,15.382,13.3
8,AEP,4.0074,8.4018,0.3770,0.34,4.2347,3.5702e+10,11.519,5.24
9,AFL,3.1507,12.7469,0.1813,1.66,10.0377,2.6166e+10,13.39,6.76


In [13]:
df_final.to_csv('../../Dataset/stats_final.csv')